In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('../data/interim/employements_mult_new.csv', sep=';')
df.head(2)

,id,position,employer,achievements,responsibilities,start_date,finish_date,position_clean,employer_clean
0,0.0,специалист,"АО ""РТК""",NaN,консультирование клиент продажа товар работа д...,2017-06-01,2019-06-01,специалист,
1,0.0,специалист,"ООО ""Евросеть-Ритейл""",NaN,консультирование клиент продажа товар работа д...,2017-05-01,2017-06-01,специалист,евросеть ритейл


In [3]:
df = df[~df['id'].isnull()].reset_index(drop=True)
df['id'] = df['id'].astype(int)

df['responsibilities'] = df['responsibilities'].fillna('')
df['achievements'] = df['achievements'].fillna('')

In [4]:
df['start_date'] = pd.to_datetime(df['start_date'], errors='coerce')
df['finish_date'] = pd.to_datetime(df['finish_date'], errors='coerce')
df['work_duration'] = (df['finish_date'] - df['start_date']).dt.days
df['position_clean'] = df['position_clean'].fillna('')
df['employer_clean'] = df['employer_clean'].fillna('')

In [5]:
# Этим можно заполнять пропуски
mean_work_duration = df['work_duration'].mean()
median_work_duration = df['work_duration'].median()

mean_work_duration, median_work_duration

(983.9361064030303, 485.0)

In [52]:
# df.head()

In [6]:
agg_df = df\
    .groupby('id')\
    .agg(
        all_work_duration=('work_duration', lambda x: list(x)),
        all_responsibilities=('responsibilities', lambda x: ' '.join(x)),
        all_positions=('position_clean', lambda x: ' '.join(x)),
        all_employers=('employer_clean', lambda x: ' '.join(x)),
        all_achievements=('achievements', lambda x: ' '.join(x)),
    )

agg_df['mean_work_duration'] = agg_df['all_work_duration'].apply(np.nanmean)
agg_df['max_work_duration'] = agg_df['all_work_duration'].apply(np.nanmax)
agg_df['min_work_duration'] = agg_df['all_work_duration'].apply(np.nanmin)
agg_df['median_work_duration'] = agg_df['all_work_duration'].apply(np.nanmedian)

agg_df = agg_df.drop(columns='all_work_duration')

In [7]:
agg_df.head()

,all_responsibilities,all_positions,all_employers,all_achievements,mean_work_duration,max_work_duration,min_work_duration,median_work_duration
id,,,,,,,,
0,консультирование клиент продажа товар работа д...,специалист специалист,евросеть ритейл мегафон ритейл мегафон,,304.750000,730.0,31.0,229.0
1,фриланс nbsp копирайтинг рерайтинг стать разли...,копирайтер рерайтер,,,NaN,NaN,NaN,NaN
2,прохождение производственный практика 2014 201...,стажёр,комитет управление муниципальный собственнос...,,30.000000,30.0,30.0,30.0
3,работа клиент оформление договор бухгалтер кас...,консультант бухгалтер старший кассир экономист,почта банк сага банк,,994.666667,1645.0,638.0,701.0
4,предоставление заявитель государственный муниц...,специалист категория старший специалист привле...,банк открытие кредит европа банк,наградить почётный грамота администрация рай...,1086.333333,1889.0,550.0,820.0


In [8]:
agg_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 343355 entries, 0 to 437527
Data columns (total 8 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   all_responsibilities  343355 non-null  object 
 1   all_positions         343355 non-null  object 
 2   all_employers         343355 non-null  object 
 3   all_achievements      343355 non-null  object 
 4   mean_work_duration    320300 non-null  float64
 5   max_work_duration     320300 non-null  float64
 6   min_work_duration     320300 non-null  float64
 7   median_work_duration  320300 non-null  float64
dtypes: float64(4), object(4)
memory usage: 23.6+ MB


In [9]:
agg_df = agg_df.fillna(0)

In [11]:
agg_df = agg_df.reset_index()

In [13]:
agg_df.to_pickle('../data/interim/employements_aggregated.pkl')